# MMDetection PIPELINE 공부용 코드

https://mmdetection.readthedocs.io/en/latest/tutorials/data_pipeline.html

In [ ]:
import numpy as np
from rich import print
from copy import deepcopy

import matplotlib.pyplot as plt
from mmdet.datasets.pipelines.transforms import Resize, Pad, Albu
from mmdet.datasets.pipelines.loading import LoadImageFromFile, LoadAnnotations
from mmdet.core.visualization import imshow_det_bboxes

In [ ]:
load = LoadImageFromFile()
loadann = LoadAnnotations(with_bbox=True)
rsz = Resize(
    img_scale=[(1333, 512), (1333, 1024)],
    keep_ratio=True
)

pad = Pad(
    size_divisor=32
)

In [ ]:
# reulsts: 데이터셋 내부에서 pipeline 호출 직전의 이미지 데이터
#
# https://bit.ly/3AMpNE4
# 위 링크에서 self.pre_pipeline 후, self.pipeline 직전의 데이터임

results = {
    'img_prefix': '/opt/ml/dataset/',
    'seg_prefix': None,
    'proposal_file': None,
    'bbox_fields': [],
    'mask_fields': [],
    'seg_fields': [],
    'img_info': {        
        'width': 1024,
        'height': 1024,
        'file_name': 'train/0000.jpg',
        'license': 0,
        'flickr_url': None,
        'coco_url': None,
        'date_captured': '2020-12-26 14:44:23',
        'id': 0,
        'filename': 'train/0000.jpg',
    },
    'ann_info': {
        'bboxes': np.array([[197.6, 193.7, 745.4, 663.4]], dtype=np.float32),
        'labels': np.array([0]),
        'bboxes_ignore': np.empty((0, 4)),
        'masks': [None],
        'seg_map': 'train/0000.png'
    }
}

ret = load(results)
ret = loadann(ret)
ret.keys()

In [ ]:
ret2 = rsz(deepcopy(ret))
print(ret2.keys())
print(ret2['img'].shape)
imshow_det_bboxes(ret2['img'], bboxes=ret2['gt_bboxes'], labels=ret2['gt_labels'])

In [ ]:
ret3 = pad(ret2)
print(ret3.keys())
print(ret3['img'].shape)
imshow_det_bboxes(ret3['img'], bboxes=ret3['gt_bboxes'], labels=ret3['gt_labels'])

In [ ]:
albu_train_transforms = [
    dict(
        type='ShiftScaleRotate',
        shift_limit=0.0625,
        scale_limit=0.0,
        rotate_limit=0,
        interpolation=1,
        p=0.5),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(
                type='RGBShift',
                r_shift_limit=10,
                g_shift_limit=10,
                b_shift_limit=10,
                p=1.0),
            dict(
                type='HueSaturationValue',
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=1.0)
        ],
        p=0.1),
    dict(type='JpegCompression', quality_lower=85, quality_upper=95, p=0.2),
    dict(type='ChannelShuffle', p=0.1),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=3, p=1.0)
        ],
        p=0.1),
]

myalbu = Albu(
    transforms=albu_train_transforms,
    bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
    keymap={
        'img': 'image',
        'gt_masks': 'masks',
        'gt_bboxes': 'bboxes'
    },
    update_pad_shape=False,
    skip_img_without_anno=True
)

In [ ]:
ret4 = myalbu(ret3)
print(ret4.keys())
print(ret4['img'].shape)
imshow_det_bboxes(ret4['img'], bboxes=ret4['gt_bboxes'], labels=ret4['gt_labels'])